In [1]:
import pandas as pd

#This code is for combining certain data from the necessary csv files into a single dataframe (complete)
pd.set_option('display.max_colwidth', None)

movies_full = pd.read_csv('newdata/movies_metadata.csv',usecols=("genres","id" ,"title","tagline", "overview","production_companies"),
                          dtype={"tagline": "string", "id":"string", 'genres':"string", "title": "string", "tagline": "string","overview":"string", "production_companies" :"string"})
ratings = pd.read_csv('newdata/ratings.csv', usecols = ("userId", "movieId", "rating"), dtype={"userId": "string","movieId": "string","rating": "string"})
ratings = ratings.rename(columns={"movieId": "id"})

keywords = pd.read_csv('newdata/keywords.csv', usecols = ("id", "keywords"), dtype={"id": "string","keywords":"string"})
credits = pd.read_csv("newdata/credits.csv", usecols = ("cast", "id"), dtype={"cast": "string", "id": "string"})

complete =  pd.merge(movies_full, ratings, on ="id")
complete =  pd.merge(complete,keywords, on ="id")
complete  = pd.merge(complete,credits, on ="id")


complete = complete.sort_values(by = 'userId')

complete  = complete.dropna()

complete  = complete.loc[:,['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview" ]]


In [2]:
import ast


#used to filter out the rows of data with empty entries
def condition(array):
    length = len(array[4])
    if(array[4][length-2:] == "[]"):
        return False
    length = len(array[5])
    if(array[5][length-2:] == "[]"):
        return False
    length = len(array[6])
    if(array[6][length-2:] == "[]"):
        return False
    length = len(array[7])
    if(array[7][length-2:] == "[]"):
        return False   
    length = len(array[8])
    if(array[8][length-4:]=="<NA>"):
        return False
    length = len(array[9])
    if(array[9][length-4:]=="<NA>"):
        return False 
    return True


#used to extract names
def populate_names(item):
    string  = item[1:-1]
    jsons = string.split("}, ")   
    names = ""
    cnt = 0
    for item in jsons:
        if(cnt == len(jsons)-1):
            temp_dict = ast.literal_eval(item)
            names+=str(temp_dict["name"])
        else:
            temp_dict = ast.literal_eval(item+"}")
            names+=str(str(temp_dict["name"])+" ")
        cnt += 1
    return names

#extract data from row of complete_array
def provide_data(array):
    movie_data = []
    movie_data.append(int(array[0]))
    movie_data.append(int(array[1]))
    movie_data.append(float(array[2]))
    movie_data.append(array[3])  

    movie_data.append(populate_names(array[4]))
    movie_data.append(populate_names(array[5]))
    movie_data.append(populate_names(array[6]))
    movie_data.append(populate_names(array[7]))

    movie_data.append(str(array[8]))
    movie_data.append(str(array[9]))
    return movie_data
    


#convert the dataframe into an array and build a dictionary
user_to_data = dict()
complete_array = complete.to_numpy()


#get all unique user ids
list_of_user_ids = []
last_id  = -1
for item in complete_array:
    if(item[0]!= last_id):
        list_of_user_ids.append(item[0])
        last_id = item[0]


index  = 0
#this has been tested with 5000, 10000, 20000, 100000
nof_users = 20000
#populate user_to_data from complete_array
for i in range(0, nof_users):
    user_to_data[list_of_user_ids[i]] = []
    for j in range(index, len(complete_array)):
        if complete_array[j][0] == list_of_user_ids[i]:
            #condition is checked for complete_array[j]
            if(condition(complete_array[j])):
                #this is where data is tranformed
                transformed = provide_data(complete_array[j])
                user_to_data[list_of_user_ids[i]].append(transformed)         
        else:
            #ignore if the number of ratings for a user is too small
            if (len(user_to_data[list_of_user_ids[i]])<10):
                del user_to_data[list_of_user_ids[i]]
            index = j+1
            break
        

In [3]:
#save in a file so that cells below can run without running this cell and above
import csv

with open("constructedData/constructedData.csv", "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview"])
    for key in user_to_data.keys():
        writer.writerows(user_to_data[key])

In [1]:
#this is a starting point if the data is already saved to the constructedData.csv file
import csv

data_list =[]

with open("constructedData/constructedData.csv", 'r', encoding="utf-8") as read_obj:
    csv_reader = csv.reader(read_obj)
    data_list = list(csv_reader)

data_list = data_list[1:]


In [2]:
from ordered_set import OrderedSet

#movie id to list of its ratings by all users
movie_to_ratings = dict()

#user id to the ratings of movies by the user
#actually includes whole data row from constucted data...
user_to_ratings = dict()

#The list created by the constructed data csv is in order by user id
#This code populates movie_to_ratings and user_to_ratings
user_id = -1
for row in data_list:
    if (row[0]!=user_id):
        user_id = row[0]
        user_to_ratings[row[0]] = [row]
    else:
        user_to_ratings[row[0]].append(row)

    if(row[1] in movie_to_ratings.keys()):
        movie_to_ratings[row[1]].append(row[2])
    else:
        movie_to_ratings[row[1]] = [row[2]]


In [3]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer

#dictionary of user id to a list of strings of combined textual features for each movie rated by the user
#the strings do not include ratings or movie id

#could this instead be user to movie id to corpus
user_to_movie_id_to_corpus = dict()


# WordNetLemmatizer().lemmatize(token.lower())
wnl = WordNetLemmatizer()


for user in user_to_ratings.keys():
    movie_strings = dict()
    for movie_data in user_to_ratings[user]:
        movie_string = ""
        #avoid the first three data points (user id, movieid, and rating)
        #use only the text data
        for index in range (3,len(movie_data)):
            if(index!= len(movie_data)-1):
                movie_string+= movie_data[index]+" "
            else:
                movie_string+= movie_data[index]
        cleaned = remove_stopwords(movie_string)
        #why recreate the string when you could accept it as a list???
        cleaned = " ".join([wnl.lemmatize(word) for word in cleaned.split(" ")])
        movie_strings[movie_data[1]] = cleaned
    user_to_movie_id_to_corpus[user] = movie_strings

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
import copy
from scipy.stats import kurtosis
from scipy.stats import skew
import statistics
import math
from ordered_set import OrderedSet
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering


#seed for consistent results across runtime
seed_int = 1
random.seed(seed_int)


#get average rating for a single movie amoung all users who rated it
def get_avg_movie_rating(movie_id):
    ret =0 
    cnt = 0
    for item in movie_to_ratings[movie_id]:
        ret+= float(item)
        cnt+=1
    return float(ret/cnt)


#get all the movie ratings from a single user
def get_user_ratings(user_id):
    ret = []
    for item in user_to_ratings[user_id]:
        ret.append(float(item[2]))
    return ret

def get_user_movie_id_to_rating(user_id):
    ret = dict()
    for item in user_to_ratings[user_id]:
        ret[int(item[1])] = float(item[2])
    return ret


#dictionary of user to list of ratings for every movie in train set, even movies not rated by the user...
#they will be marked with a rating of -1
#the empty ratings will be filled with a similair process to below (taking the rating of the most simlair user)
#this must be done before clustering

user_to_movie_id_to_rating = dict()
user_to_rand_movie_id = dict()
user_to_rand_words_to_counts = dict()
user_to_movie_id_to_words_to_counts = dict()
user_to_movie_id_to_corpus_test = dict()
movie_id_to_words_to_count = dict()
words_in_order = OrderedSet()
#how does copy work here???
user_to_movie_id_to_corpus_train = user_to_movie_id_to_corpus.copy()


for i in range(50):
    user = random.choice(list(user_to_movie_id_to_corpus_train.keys()))
    user_to_movie_id_to_corpus_test[user] = user_to_movie_id_to_corpus_train[user]
    user_to_movie_id_to_corpus_train.pop(user)


for user in user_to_movie_id_to_corpus_train.keys():
    #copy needed???
    movie_id_to_rating = get_user_movie_id_to_rating(user)
    user_to_rand_movie_id[user] = random.choice(list(user_to_movie_id_to_corpus_train[user].keys()))
    temp = dict()
    for movie_id in movie_id_to_rating.keys():
        if movie_id != user_to_rand_movie_id[user]:
            if movie_id in movie_id_to_rating.keys():
                temp[movie_id] = movie_id_to_rating[movie_id]
            else:
                temp[movie_id] = -1
    user_to_movie_id_to_rating[user] = temp


#note: the words should be stored as a list and sorted before this and the next process
for user in user_to_movie_id_to_corpus_train.keys():
    for movie_id in user_to_movie_id_to_corpus_train[user].keys():
        if movie_id != user_to_rand_movie_id[user]:
            for word in user_to_movie_id_to_corpus_train[user][movie_id].split(" "):
                words_in_order.add(word)

            
for user in user_to_movie_id_to_corpus_train.keys():
    user_to_movie_id_to_words_to_counts[user] = dict()
    for movie_id in user_to_movie_id_to_corpus_train[user].keys():
        if movie_id != user_to_rand_movie_id[user]:
            temp = dict()
            for word in words_in_order:
                temp[word] = user_to_movie_id_to_corpus_train[user][movie_id].count(word)
            user_to_movie_id_to_words_to_counts[user][movie_id] = temp
            if movie_id not in movie_id_to_words_to_count.keys():
                movie_id_to_words_to_count[movie_id] = temp
        else:
            temp = dict()
            for word in words_in_order:
                temp[word] = user_to_movie_id_to_corpus_train[user][movie_id].count(word)
            user_to_rand_words_to_counts[user] = temp


#what if a new user to be tested has new words that are new to words_in_order???
#what if a user simply has more words than normal, need to normalize(not this is doen with cossine similairity)
#what if a user rated a movie that has not been rated by any users??? (is it omited ???)
#do the randomly selected users meed to be omited from the words_in_order and user_to_list_of_words_to_counts
#to emulate completely new users???


def predict(user, words_to_counts):
    #what kind of copy is needed???
    #copy.deepcopy???
    temp_movie_id_to_words_to_counts = user_to_movie_id_to_words_to_counts[user].copy()

    values = []
    for movie_id in temp_movie_id_to_words_to_counts.keys():
        values.append(temp_movie_id_to_words_to_counts[movie_id].values())
    
    cosine_sim = cosine_similarity(X = values ,Y = [words_to_counts.values()])
    #not sure if reshape is needed???
    cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))

    ratings = []
    for movie_id in user_to_movie_id_to_rating[user].keys():
        ratings.append(user_to_movie_id_to_rating[user][movie_id])


    max = -10
    pred_rating = ratings[0]

    #note: for accuracy purposes, the user should have a certain number of rated movies
    #instead of selecting the most similair movie should clusters be used???
    #for now this simple method will suffice
    #are the ratings lined up with the cossine similarities???
    #yes the order of user_to_ratings is used to make user_to_corpus_list...
    #which is used to make lst_of_word_to_nof_occurances

    for sim, rat in zip(cosine_sim, ratings):
        if sim>max:
            max = sim
            pred_rating = rat

    return pred_rating


#fill in the missing ratings for a user
def fill_in_ratings(user):
    for movie_id in user_to_movie_id_to_rating[user].keys():
        if user_to_movie_id_to_rating[user][movie_id] == -1:
            user_to_movie_id_to_rating[user][movie_id] = predict(user, movie_id_to_words_to_count[movie_id])





#now for the user comparison logic (need user to list of movie ratings)
#fill in ratings that the user hasn't watched with the method above
#then cluster the users by their ratings

#note: agglomerative clustering might make more sense here since k-means has random init for centroids...
#note: to guess a new users rating requires that none of that users ratings have been used to train the model
#The data needs to be split into test and train before modeling the algorithm on the train data

#Training process:
#split data into test and train data
#proceed with train data...
#cluster movies by the tokens with range for k
#cluster users by the ratings with range for k and (fill in ratings for movies a users hasn't watched with some guess)
#guess: this can be obtained by clustering the movies that the user has watched...
#for each movie the user hasn't watched find the cluster that it belongs to with the highest possible k value
#that the user has at least one movie belonging to one of the clusters and then take the average of those movies
#this is exactly like a later training step excpet it is applied to all the movies the user watched

#for a single randomly chosen movie from each user in the trainging data...

#find the cluster the movie belongs to 
#find the movies part of that same cluster that the user has scored at the highest possible k value
#take the average score of these movies
#find the cluster the user belongs to
#find the average rating of the movie for users in that cluster at the highest possible k value
#train an mlp model with both averages and perhaps some extra statistics as features...
#using the given movie ratings as actuals


#The process of predicting a rating:
#1. find the cluster the movie belongs to 
#2. find the movies part of that same cluster that the user has scored at the highest possible k value
#3. take the average score of these movies
#4. find the cluster the user belongs to
#5. find the average rating of the movie for users in that cluster at the highest possible k value
#6. input into the trained mlp model both averages and perhaps some extra statistics
#7. make predictions and test against the randomly chosen movies actual ratings


#summary:
#find cluster for movie -> find movies part of the same clusters that the users rated -> average
#question: are the clusters unique to the movies the user has watched or to all movies???
#what is the technical difference???
#is this the same as finding the most simimlair movie the user rated and copying the rating???

#find cluster for user -> find the ratings for the movie by people in the same cluster -> average

#other avenues considered:
#idea 1:
#for the first process, instead of averaging the movies that only the user rated, find other users that are...
#like the user in question and find the average for that movie cluster
#Problem: it is better to get the users raw opionion rather than generalizing it to some like minded users
#there is an extra costly step to this
#idea 2: 
#for the second process, instead of finding the average rating for the movie in the same cluster of users...
#also find the average rating of movies that are like the movie in question 
#Problem, it is better to get the movies rating itself as it would be the most accurate indicator
#there is an extra costly step to this

MemoryError: 

: 